In [ ]:
from gs_quant.instrument import FXOption
from gs_quant.risk import FXAnnualImpliedVol, Price, FXSpot
from gs_quant.session import GsSession, Environment
from gs_quant.common import MeasureScenario
from gs_quant.markets import PricingContext
import datetime as dt

In [69]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(Environment.PROD, client_id=None, client_secret=None, scopes=('run_analytics',))

MeasureScenario class allows us to override inputs such as spot or volatility when pricing an instrument. Here are some examples of how to use it with FX Options.

In [ ]:
# Initial FX Option
fx_option = FXOption(pair='EURUSD', expiration_date='3m', premium=0)
option = fx_option.calc((Price, FXAnnualImpliedVol, FXSpot))
option.to_frame()

In [ ]:
# Override a single input (Spot/Strike Price) with MeasureScenario context manager
with MeasureScenario(FXSpot, 1.19):
    fx_risks = fx_option.calc((Price, FXAnnualImpliedVol, FXSpot))
fx_risks.to_frame()

We can see that the FXSpot value was overridden, and other variables are re-calculated to account for the new spot rate.

In [ ]:
# Override multiple inputs (Spot rate and Price) with MeasureScenario context manager
with MeasureScenario(FXSpot, 1.19):
    with MeasureScenario(FXAnnualImpliedVol, 0.05):
        fx_risks = fx_option.calc((Price, FXAnnualImpliedVol, FXSpot))
fx_risks.to_frame()

We can also combine the measure override with other context managers, such as PricingContext

In [ ]:
# Set up time context for pricing, and override the input
with PricingContext(dt.date(2025, 7, 5)):
    with MeasureScenario(FXSpot, 2):
        fx_option_risks = fx_option.calc((Price, FXAnnualImpliedVol, FXSpot))
fx_option_risks.result().to_frame()